# M3 Data collection and management : project
## Planning my next holidays ☀️

Let's put your dataset in a postgreSQL database on Amazon RDS ! Then, you can use the SQL syntax to make requests and decide where you'll go for your holidays 😎

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import boto3

1. Read the .csv files containing information about cities and hotels

In [2]:
df_weather = pd.read_csv('weather_cities.csv', index_col = 0)
df_weather.head()

,city_id,city_name,lat,lon,temperature,main_weather,humidity,expected_rain,wind_speed,UV_indice,rank,inverted_rank
0,33,Bayonne,43.493338,-1.475099,18.51,Clouds,58.43,0.5712,1.68,1.75,1,35
1,32,Biarritz,43.471144,-1.552727,17.88,Clouds,61.00,0.6106,1.73,1.74,2,34
2,27,Collioure,42.525050,3.083155,17.51,Clear,64.29,0.1631,2.40,1.82,3,33
3,15,Grenoble,45.187560,5.735782,17.32,Clouds,48.86,0.0000,0.86,1.56,4,32
4,19,Cassis,43.214036,5.539632,17.25,Clear,61.71,1.5052,2.15,1.76,5,31


In [3]:
df_list_hotels = pd.read_csv('list_hotels.csv', index_col = 0)
df_list_hotels.head()

,city_id,city_name,hotel id,hotel name,hotel url,latitude,longitude,score,description
0,33,Bayonne,0,Hôtel Le Bayonne,https://www.booking.com/hotel/fr/le-bayonne-ha...,43.485026,-1.475371,8.1,"Installé dans le centre-ville de Bayonne, en ..."
1,33,Bayonne,1,ibis budget Bayonne,https://www.booking.com/hotel/fr/ibis-budget-b...,43.489083,-1.457178,7.7,L'hôtel ibis budget Bayonne est situé à 2 km ...
2,33,Bayonne,2,Okko Hotels Bayonne Centre,https://www.booking.com/hotel/fr/okko-hotels-b...,43.495344,-1.482512,8.8,"Situé à Bayonne, à seulement 700 mètres du ce..."
3,33,Bayonne,3,Hotel Cote Basque,https://www.booking.com/hotel/fr/cote-basque.f...,43.496183,-1.469601,8.1,L'Hotel Cote Basque est situé au cœur de la v...
4,33,Bayonne,4,Hôtel Villa KOEGUI Bayonne,https://www.booking.com/hotel/fr/villa-koegui-...,43.492371,-1.472588,8.9,"Doté d’un bar, d’un restaurant, d’une terrass..."


2. Create and configure a RDS instance in your AWS account. 

3. Use SQLAlchemy to create an engine that is connected to the remote database

In [5]:
from sqlalchemy import create_engine
engine = create_engine("postgresql+psycopg2://postgres_1986:****@"
                       "****/****", echo=True)

4. Create two tables in your remote database : `cities` and `hotels`, each one containing the information from the different .csv files you saved yesterday

In [6]:
df_weather.to_sql('cities_weather', engine)

2020-11-11 11:44:28,916 INFO sqlalchemy.engine.base.Engine select version()
2020-11-11 11:44:28,918 INFO sqlalchemy.engine.base.Engine {}
2020-11-11 11:44:28,927 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-11-11 11:44:28,928 INFO sqlalchemy.engine.base.Engine {}
2020-11-11 11:44:28,939 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-11-11 11:44:28,940 INFO sqlalchemy.engine.base.Engine {}
2020-11-11 11:44:28,947 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-11-11 11:44:28,947 INFO sqlalchemy.engine.base.Engine {}
2020-11-11 11:44:28,952 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-11-11 11:44:28,952 INFO sqlalchemy.engine.base.Engine {}
2020-11-11 11:44:28,962 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [7]:
df_list_hotels.to_sql('hotels', engine)

2020-11-11 11:44:50,616 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-11-11 11:44:50,617 INFO sqlalchemy.engine.base.Engine {'name': 'hotels'}
2020-11-11 11:44:50,639 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE hotels (
	index BIGINT, 
	city_id BIGINT, 
	city_name TEXT, 
	"hotel id" BIGINT, 
	"hotel name" TEXT, 
	"hotel url" TEXT, 
	latitude FLOAT(53), 
	longitude FLOAT(53), 
	score FLOAT(53), 
	description TEXT
)


2020-11-11 11:44:50,640 INFO sqlalchemy.engine.base.Engine {}
2020-11-11 11:44:50,655 INFO sqlalchemy.engine.base.Engine COMMIT
2020-11-11 11:44:50,661 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_hotels_index ON hotels (index)
2020-11-11 11:44:50,662 INFO sqlalchemy.engine.base.Engine {}
2020-11-11 11:44:50,674 INFO sqlalchemy.engine.base.Engine COMMIT
2020-11-11 11:44:50,681 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
202

5. Optional : use PGAdmin to explore the database

6. Use SQL syntax to make a request that will help you get the list of the 30 best places to go (among all cities and hotels) 😎

In [13]:
from sqlalchemy.sql import text

conn = engine.connect()

stmt_30_best_places = text('SELECT cw.city_name, cw.main_weather, cw.temperature, '
                          'h."hotel name", h.score, h.description, h."hotel url" '
                          'FROM cities_weather as cw, hotels as h '
                          'WHERE cw.city_id = h.city_id '
                          'ORDER BY score DESC, temperature DESC '
                          'LIMIT 30'
                          )
                          # OR fetch first 30 rows only

res_30_best_place = conn.execute(stmt_30_best_place)
res_30_best_place.fetchall()              

2020-11-11 12:30:42,764 INFO sqlalchemy.engine.base.Engine SELECT cw.city_name, cw.main_weather, cw.temperature, h."hotel name", h.score, h.description, h."hotel url" FROM cities_weather as cw, hotels as h WHERE cw.city_id = h.city_id ORDER BY score DESC, temperature DESC LIMIT 30
2020-11-11 12:30:42,765 INFO sqlalchemy.engine.base.Engine {}


[('Biarritz', 'Clouds', 17.88, ' Villa Uhaïna ', 10.0, ' La Villa Uhaïna est située sur le front de mer à Biarritz, à 1,1 km de la plage de la Côte des Basques et de celle de Marbella, dans une région propice à la randonnée. ', 'https://www.booking.com/hotel/fr/villa-milady.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaE2IAQGYAQ24AQrIAUjYAQHoAQH4AQKIAgGoAgO4Apjbq_0FwAIB0gIkY2ZjZW ... (173 characters truncated) ... dren=0&hapos=402&hpos=2&no_rooms=1&sr_order=popularity&srepoch=1605037487&srpvid=17c68ad7938a0003&ucfs=1&from=searchresults;highlight_room=#hotelTmpl'),
 ('Bayonne', 'Clouds', 18.51, ' T2 au coeur de Bayonne ', 9.9, ' Offrant une vue sur la ville, le T2 au cœur de Bayonne est un hébergement situé à Bayonne, à 400 mètres de la cathédrale Sainte-Marie et à 1,2 km de la plaine d’Ansot. ', 'https://www.booking.com/hotel/fr/t2-au-coeur-de-bayonne.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaE2IAQGYAQ24AQrIAUjYAQHoAQH4AQKIAgGoAgO4Aryrqf0FwAIB ... (155 characters truncated) ... dren=0&hapo

In [12]:
df_30_best_places = pd.read_sql(stmt_30_best_places, engine)
df_30_best_places

2020-11-11 12:28:10,578 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-11-11 12:28:10,579 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT cw.city_name, cw.main_weather, cw.temperature, h."hotel name", h.score, h.description, h."hotel url" FROM cities_weather as cw, hotels as h WHERE cw.city_id = h.city_id ORDER BY score DESC, temperature DESC LIMIT 30'}
2020-11-11 12:28:10,604 INFO sqlalchemy.engine.base.Engine SELECT cw.city_name, cw.main_weather, cw.temperature, h."hotel name", h.score, h.description, h."hotel url" FROM cities_weather as cw, hotels as h WHERE cw.city_id = h.city_id ORDER BY score DESC, temperature DESC LIMIT 30
2020-11-11 12:28:10,604 INFO sqlalchemy.engine.base.Engine {}


,city_name,main_weather,temperature,hotel name,score,description,hotel url
0,Biarritz,Clouds,17.88,Villa Uhaïna,10.0,La Villa Uhaïna est située sur le front de me...,https://www.booking.com/hotel/fr/villa-milady....
1,Bayonne,Clouds,18.51,T2 au coeur de Bayonne,9.9,"Offrant une vue sur la ville, le T2 au cœur d...",https://www.booking.com/hotel/fr/t2-au-coeur-d...
2,Biarritz,Clouds,17.88,Bellevue,9.8,"Le Bellevue est situé à Biarritz, à 500 mètre...",https://www.booking.com/hotel/fr/bellevue-biar...
3,Biarritz,Clouds,17.88,"L'Etape, 2ch, Parking, 250m Gde Plage, tt à p...",9.8,"L'Etape, 2ch, Parking met gratuitement à votr...",https://www.booking.com/hotel/fr/etape-2ch-par...
4,Biarritz,Clouds,17.88,JUST LIKE HOME BIARRITZ - Private Parking - F...,9.8,"Offrant une vue sur le jardin, le JUST LIKE H...",https://www.booking.com/hotel/fr/appartement-s...
5,Collioure,Clear,17.51,Les mimosas,9.8,Doté d’un jardin et d’une piscine extérieure ...,https://www.booking.com/hotel/fr/les-mimosas-c...
6,Collioure,Clear,17.51,Boramar,9.8,"Situé à Collioure, à quelques pas de la plage...",https://www.booking.com/hotel/fr/boramar.fr.ht...
7,Collioure,Clear,17.51,Les Hauts de Collioure,9.8,Doté d'une piscine extérieure ouverte en sais...,https://www.booking.com/hotel/fr/les-hauts-de-...
8,Bayonne,Clouds,18.51,Duplex avec belle térasse au centre de Bayonne,9.7,Duplex avec belle térasse au centre de Bayonn...,https://www.booking.com/hotel/fr/duplex-avec-b...
9,Bayonne,Clouds,18.51,UNIQUE APPARTEMENT TYPE HAUSSMANIEN - BAYONNE...,9.7,"Donnant sur la rivière, l’UNIQUE APPARTEMENT ...",https://www.booking.com/hotel/fr/unique-appart...
